# 0. パッケージのインポート

In [253]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 1. 初期値の設定

滴定実験の詳細を決める。以下に従って **下の欄に所望の値を代入し、セルを実行** せよ。

- 初期総濃度の指定
    - `ca0`: 酸 $\mathrm H_m\mathrm A $の初期総濃度 $C_{\text A}^{(0)}$
    - `cb0`: 塩基 $\mathrm B(\mathrm{OH})_n$ の初期総濃度 $C_{\text B}^{(0)}$

- 総体積の指定
    - `va0`: 酸 $\mathrm H_m\mathrm A$ の総体積 $V_{\text A}^{(0)}$
    - `vb0`: 塩基 $\mathrm B(\mathrm{OH})_n$ の総体積 $V_{\text B}^{(0)}$

- 滴下実験の実施
    - `drip`: 酸 $\mathrm H_m\mathrm A$ を滴下するなら`'a'`、塩基 $\mathrm B(\mathrm{OH})_n$ を滴下するなら`'b'`
    - `drip_step`: 滴下回数

In [257]:
ca0 = 1.0e-2
cb0 = 1.0e-2

va0 = 2.0e-2
vb0 = 1.0e-2

drip = 'a'
drip_step = 1000

# 2. 平衡定数の指定

平衡定数は **別ファイルで指定** する。`acid.dis`には、上から順に平衡定数 $K_{\text a1}, K_{\text a2},\dots$ を記録せよ。

$$
\begin{aligned}
    \mathrm H_{m\phantom{{}-0}}\mathrm A^{\phantom{-}} &\stackrel{K_{\text a1}}{\rightleftarrows} \mathrm H^+ + \mathrm H_{m-1}\mathrm A^{\phantom{1}-}\\
    \mathrm H_{m-1}\mathrm A^- &\stackrel{K_{\text a2}}{\rightleftarrows} \mathrm H^+ + \mathrm H_{m-2}\mathrm A^{2-}\\
    &\hspace{0.75em}\vdots
\end{aligned}
$$

`base.dis`には、上から順に平衡定数 $K_{\text b1}, K_{\text b2},\dots$ を記録せよ。

$$
\begin{aligned}
    {\mathrm B(\mathrm{OH})_{n-1}}^{\phantom{1}+} + \mathrm{H_2O} &\stackrel{K_{\text b1}}{\rightleftarrows} {\mathrm B(\mathrm{OH})_{n\phantom{{}-0}}}^{\phantom{+}} + \mathrm H^+\\ 
    {\mathrm B(\mathrm{OH})_{n-2}}^{2+} + \mathrm{H_2O} &\stackrel{K_{\text b2}}{\rightleftarrows} {\mathrm B(\mathrm{OH})_{n-1}}^+ + \mathrm H^+\\ 
    &\hspace{0.75em}\vdots
\end{aligned}
$$

上の式で定義される塩基の平衡定数 $K_{\text bi}$ がわからなければ、以下の変換公式が有効である。
$$ K_{\text bi}=\frac{K_{\text w}}{K_{\text bi}'},\quad {\mathrm B(\mathrm{OH})_{n-i+1}}^{(i-1)+}\stackrel{K_{\text bi}'}{\rightleftarrows} {\mathrm B(\mathrm{OH})_{n-i}}^{i+}+\mathrm{OH}^- $$

# 3. 実行

以上のセルの実行が正常にできれば、次のセルを実行して中和滴定曲線が得られる。
`out.dat`に実行結果が記録され、セルの下にグラフが表示される。

In [ ]:
kw = 1.008e-14

def p_inv(x):
    return 10**(-x)

def read_acidbase(file_acid: str, file_base: str):
    global kw
    
    acid_arr = np.insert(np.genfromtxt(file_acid, dtype=float), 0, 1.)
    base_arr = np.insert(np.genfromtxt(file_base, dtype=float), 0, 1.)

    if type(acid_arr) != np.ndarray:
        acid_arr = np.array([acid_arr])
    if type(base_arr) != np.ndarray:
        base_arr = np.array([base_arr])
    return acid_arr, base_arr

def equation(x, ca: float, cb: float):
    global acid_arr, base_arr, kw

    m, n = len(acid_arr), len(base_arr)

    nmr_a, dnm_a, nmr_b, dnm_b = 0., 0., 0., 0.
    for i in range(m+1):
        nmr_a += i * np.prod(acid_arr[0:i+1]) / p_inv(x) ** i
        dnm_a +=     np.prod(acid_arr[0:i+1]) / p_inv(x) ** i
    for j in range(n+1):
        nmr_b += j * p_inv(x) ** j / np.prod(base_arr[0:j+1])
        dnm_b +=     p_inv(x) ** j / np.prod(base_arr[0:j+1])

    a = ca * nmr_a / dnm_a - cb * nmr_b / dnm_b
    b = -kw
    ans = p_inv(x) - (a + np.sqrt(a ** 2 - 4 * b)) / 2
    return np.log10(np.abs(ans))

def solve(ca: float, cb: float):
    global equation, acid_arr, base_arr, kw

    x_arr = np.linspace(0., 15., 1500, endpoint=False)
    min = 0.
    for x in x_arr:
        y = equation(x, ca, cb)
        if y < min:
            x_guess = x
            min = y
    x_guess = minimize(equation, x_guess, args=(ca, cb), method='Powell', tol=1e-12).x[0]
    return x_guess

def run(drip: str, step: int):
    global ca0, cb0, va0, vb0, kw
    h_arr = []

    if drip == 'a':
        v_arr = np.linspace(0., va0, step, endpoint=False)
        h = -np.log10(cb0)
        
        for va in v_arr:
            ca = (ca0 * va ) / (va + vb0)
            cb = (ca0 * vb0) / (va + vb0)

            h = solve(ca, cb)
            h_arr.append(h)
    elif drip == 'b':
        v_arr = np.linspace(0., vb0, step, endpoint=False)
        h = -np.log10(ca0)
        
        for vb in v_arr:
            ca = (ca0 * va0) / (va0 + vb)
            cb = (ca0 * vb ) / (va0 + vb)

            h = solve(ca, cb)
            h_arr.append(h)
    else:
        raise ValueError(f'run() received an invalid argument: {drip}. The argument should be either \'a\' or \'b\'.')
    
    return [[v_arr[i], h] for i, h in enumerate(h_arr)]

def plot(results: list):
    global drip, va0, vb0
    x = [result[0] for result in results]
    y = [result[1] for result in results]

    plt.plot(x, y, linestyle='-', color='b')
    plt.grid(True)
    plt.xlabel('Dripped volume v /L')
    plt.ylabel('pH')
    if drip == 'a':
        plt.xlim(0,va0)
    elif drip == 'b':
        plt.xlim(0,vb0)
    else:
        raise ValueError(f'plot() received an invalid argument: {drip}. The argument should be either \'a\' or \'b\'.')

    plt.ylim(0,14)

    plt.show()

acid_arr, base_arr = read_acidbase('acid.dis', 'base.dis')
results = run(drip, drip_step)

with open('out.dat', 'w') as f:
    for res in results:
        f.write(f'{res[0]}\t{res[1]}\n')
plot(results)

## 原理

電気中性の法則から得る $[\mathrm H^+]$ の多項式
$$ [\mathrm H^+]^2 - \left(C_{\text A}\cdot\frac{\displaystyle\sum_{i=0}^m\,i\cdot\frac{K_{\text a1}\cdots K_{\text ai}}{[\mathrm H^+]^i}}{\displaystyle\sum_{i=0}^m\phantom{\,i\cdot{}}\frac{K_{\text a1}\cdots K_{\text ai}}{[\mathrm H^+]^i}}-C_{\text B}\cdot\frac{\displaystyle\sum_{i=0}^n\,i\cdot\frac{[\mathrm H^+]^i}{K_{\text b1}\cdots K_{\text bi}}}{\displaystyle\sum_{i=0}^n\phantom{\,i\cdot{}}\frac{[\mathrm H^+]^i}{K_{\text b1}\cdots K_{\text bi}}}\right)[\mathrm H^+] - K_{\text w} = 0 $$
を、形式的に $[\mathrm H^+]$ の2次方程式とみなし解の公式を得る。
$$ [\mathrm H^+] = \frac{a+\sqrt{a^2-4b}}2,\quad a=\left(C_{\text A}\cdot\frac{\displaystyle\sum_{i=0}^m\,i\cdot\frac{K_{\text a1}\cdots K_{\text ai}}{[\mathrm H^+]^i}}{\displaystyle\sum_{i=0}^m\phantom{\,i\cdot{}}\frac{K_{\text a1}\cdots K_{\text ai}}{[\mathrm H^+]^i}}-C_{\text B}\cdot\frac{\displaystyle\sum_{i=0}^n\,i\cdot\frac{[\mathrm H^+]^i}{K_{\text b1}\cdots K_{\text bi}}}{\displaystyle\sum_{i=0}^n\phantom{\,i\cdot{}}\frac{[\mathrm H^+]^i}{K_{\text b1}\cdots K_{\text bi}}}\right),\quad b=-K_{\text w} $$
$ [\mathrm H^+] = \frac{a+\sqrt{a^2-4b}}2 $ は右辺が $[\mathrm H^+]$ に依るため、一種の自己無撞着方程式になっており、
$$ f([\mathrm H^+]) = \log\left|[\mathrm H^+] - \frac{a+\sqrt{a^2-4b}}2\right| $$
が最小になる $[\mathrm H^+]$ を`Powell`法で探索して求める。

総濃度 $C_{\text A}, C_{\text B}$ は滴下する溶液に依って場合分けされる。
- 塩基を酸で滴定するなら、滴下体積を$v$として、$$ C_{\text A} = \frac{v}{v+V_{\text B}^{(0)}},\quad C_{\text B} = \frac{V_{\text B}^{(0)}}{v+V_{\text B}^{(0)}} $$
- 酸を塩基で滴定するなら、滴下体積を$v$として、$$ C_{\text A} = \frac{V_{\text A}^{(0)}}{V_{\text A}^{(0)}+v},\quad C_{\text B} = \frac{v}{V_{\text A}^{(0)}+v} $$